# Ubiquant Market Prediction
投資の収益率を予測するモデルを構築<br>

- **row_id**
- **time_id** - 0~1219
- **investment_id** - 0~3773
- **target** - The target.
- **features** - [f_0:f_299] - 市場データより生成された特徴量

データの確認 ～ 提出コードの確認まで行っていきます。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc 
import warnings
warnings.filterwarnings('ignore')

### 学習データ読み込み
提供されているCSV学習データの容量は18Gほどあります。そのままでは大き過ぎるので、Daskで読み込むかpickleやparquetで扱うことになりそうです。<br>
今回は、ROB MULLAさんのParquetデータを使用させていただきます。感謝です！<br>
参考カーネル：[⏫ Fast Data Loading and Low Mem with Parquet Files](https://www.kaggle.com/robikscube/fast-data-loading-and-low-mem-with-parquet-files)

In [ ]:
%%time
df_train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
df_train

In [ ]:
df_train.info()

In [ ]:
df_train.loc[:, 'row_id':'f_3'].describe()

データ数を確認してみます。欠損ありますね。<br>
time_idについては、idが大きいほどデータがそろっている、また300後半でかなりデータの抜けがあります。<br>
investment_idは、bin数を増やすとボコボコしてるのが確認できました。

In [ ]:
f, ax = plt.subplots(3,1,figsize=(10,10))
sns.distplot(df_train['target'], ax=ax[0], bins=200, kde=False)
ax[0].set_title('Histgram of target')

sns.distplot(df_train['time_id'], ax=ax[1], bins=200, kde=False)
ax[1].set_title('Histgram of time_id')

sns.distplot(df_train['investment_id'], ax=ax[2], bins=200, kde=False)
ax[2].set_title('Histgram of investment_id')

plt.tight_layout()

### 提出確認
ubiquant Python moduleを使用する必要があるとのこと。簡易モデルで提出の確認だけしておきます。<br>
[Overview/Evaluation](https://www.kaggle.com/c/ubiquant-market-prediction/overview/evaluation)に記載の通り、predictionを行います。

In [ ]:
df_u300 = df_train[df_train.time_id<=300]
df_o1100 = df_train[df_train.time_id>=1100]
del df_train

features = ['f_'+str(i) for i in range(300)]
target = 'target'

# 学習に使用するデータを設定
data_train = lgb.Dataset(df_u300[features], df_u300[target])
data_eval = lgb.Dataset(df_o1100[features], df_o1100[target], reference=data_train) 

# LightGBM parameters
params = {
        "objective": "regression",
        "metric": "rmse",
        "num_leaves": 14,
        "max_depth": 4,
        "feature_fraction": 0.8,
        "subsample_freq": 1,
        "bagging_fraction": 0.7,
        "min_data_in_leaf": 10,
        "learning_rate": 0.2,
        "boosting": "gbdt",
        "lambda_l1": 0.4,
        "lambda_l2": 0.4,
        "verbosity": -1,
        "random_state": 42,
}

# モデルの学習
model = lgb.train(params,
                  train_set=data_train, # トレーニングデータの指定
                  valid_sets=data_eval, # 検証データの指定
                  )
df_feature = pd.DataFrame({
    'features' :features,
    'importance' : model.feature_importance()})
df_feature = df_feature.sort_values(by='importance',ascending = False)

plt.figure(figsize=(16, 6))
sns.barplot(x='features', y='importance', data=df_feature[0:30])
plt.title(f'Head LightGBM Feature Importance')
plt.tight_layout()
plt.show()

In [ ]:
import ubiquant

env = ubiquant.make_env()  # initialize the environment
iter_test = env.iter_test()  # an iterator which loops over the test set and sample submission

for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df['target'] = model.predict(test_df[features])  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions